<a href="https://colab.research.google.com/github/paulo01012018/iris-facens/blob/master/Copy_of_DataH_segundo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Segundo exemplo - Localizando Clusters com K-Means**

Adaptado do trabalho de  [Philip Kalinda](http://www.philipkalinda.com/ds3.html).

O notebook original pode ser encontrado no github neste [endereço](https://github.com/philipkalinda/data_science/blob/master/multiclass_logistic_classification_and_kmeans_clustering_of_iris_dataset/multiclass_logistic_classification_and_kmeans_clustering_of_iris_dataset.ipynb)

\
**Descrição**

Nesta análise, continuaremos a exploração do Iris Dataset.
E, em especial, este dataset pode ser explorado de diferentes formas, dependendo dos objetivos almejados quando se inicia a análise.

\
**Objetivos**

Para que a nossa análise seja eficiente, é importante definir um escopo e objetivos antes de realizar qualquer análise tendo apenas algumas poucas perguntas para nos guiar, sob a pena de nos perdermos nos dados ou nos desviarmos para análises que não acrescentam valor às nossas metas.
Assim, selecionamos os seguintes objetivos:

1.       Verificar agrupamentos identificáveis nos dados
2.       Avaliar a precisão destes agrupamentos em função das espécies.
3.       Criar um modelo para predizer as espécies dos exemplos
4.       Avaliar a acurácia do modelo preditivo
5.       Avaliar a capacidade do modelo em prever cada espécie
6.       Avaliar a acurácia do modelo em predizer dados exemplo.

\
**Perguntas a Pesquisar**

Existem algumas perguntas que procuraremos responder para atingir os objetivos, conforme apresentado a seguir:

1.       Qual das abordagens é a mais precisa?
2.       Como cada variável interage com as demais?
3.       Como cada variável influencia na predição da espécie?
4.       Qual espécie é mais fácil de ser prevista? Porque?
5.       E qual espécie é mais difícil de ser prevista? Porque?


Estas serão as bibliotecas que vamos utilizar ao longo deste texto:

\
O **Sklearn** é "O" pacote quando pensamos em criar soluções de machine learning em python. 

O **Pandas** é um pacote fenomenal para tratar de conjuntos de dados, e possui muitas funções úteis. 

O **NumPY** é o pacote para funções de álgebra linear e possui algumas funções e métodos úteis. 

O **Matplotlib** é o pacote para construir visualizações. 

E, finalmente, o **Seaborn** torna as visualizações mais interessantes e bonitas.

In [1]:
#Preparacao do ambiente
#Bibliotecas que utilizaremos;
from sklearn import datasets
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.cross_validation import train_test_split, KFold, cross_val_score
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ModuleNotFoundError: ignored

------------------------------------

Carregadas as bibliotecas, vamos aos dados!

In [0]:
#Definicao das especies
def species_label(theta):	
  return raw_data.target_names[theta]	

In [0]:
raw_data = datasets.load_iris()
data_desc = raw_data.DESCR
data = pd.DataFrame(raw_data.data, columns=raw_data.feature_names)
data['species'] = [species_label(theta) for theta in raw_data.target]
data['species_id'] = raw_data.target
data.head()

NameError: ignored

Inicialmente, um pouco de limpeza nos dados é necessária.
Uma vez que os dados tenham sido carregados para o dataframe, precisamos de uma coluna adicional conter o texto que descreve a espécie de forma que cada linha possa ser melhor identificada. Isto foi feito com o uso da função (“species_label()”) definida anteriormente.

-----------------------

**Descrever**

Para nos tornarmos mais íntimos do dataset e começar a pavimentar a estrada que nos levará ao destino, vale a pena recuperar algumas análises estatísticas descritivas sobre os dados. 
Isto é simples de se executar, mas mesmo assim, agrega muito. valor. O conteúdo retornado é apresentado abaixo:


In [0]:
data.describe()

NameError: ignored

A tabela sumariza todas as colunas numéricas do dataset. É particularmente útil para se obter uma visão geral dos dados e sobre como eles estão estruturados. 
Por exemplo, observando o intervalo e os quartis, nós podemos identificar que o comprimento da pétala é a coluna com a maior variação e a largura da sépala a que tem menor variação.
Isto pode ser útil quando nos aproximarmos do estágio da compreensão de como cada uma das variáveis afeta o modelo preditivo.

-----------------------


**Pivot Table**

O ponto central da nossa análise está relacionado às espécies. Para desenvolver a compreensão de como cada uma das variáveis se comporta em cada espécie, nós podemos produzir uma Pivot Table com as médias.


In [0]:
data.pivot_table(index='species', values=['sepal length (cm)',
                                          'sepal width (cm)',
                                          'petal length (cm)',
                                          'petal width (cm)',
                                          'species_id'],aggfunc=np.mean)

Nós podemos deduzir a partir desta tabela que, na média, o comprimento e largura das pétadas da Setosa são muito menores que da Versicolor e Virginica. Da mesma forma, a Virginica tem na média as maiores dimensões de pétadas. Nós também podemos ver que na média, os comprimentos das sépalas são muito próximos, contudo, a Setosa tem claramente larguras de sépalas maiores que das outras espécies. Ou seja, esta tabela nos mostra que a espécie Setosa tem várias características distintas.

-----------------------


**Correlação**

Uma outra forma de obter uma compreensão mais profunda da dinâmica das variáveis é através de uma tabela de correlação.
Esta tabela identifica tendências entre as variáveis e assinala para estas um número entre -1 e 1, dependendo da intensidade da relação entre elas.

\
**É importante notar que esta tabela meramente identifica tendências e não identifica a causa desta tendência nem indica que uma variável tem impacto sobre outra.**

In [0]:
d_corr=data.iloc[:,[0,1,2,3,5]].corr()

In [0]:
d_corr

Esta tabela mostra que existe uma forte relação linear entre todas as variáveis, exceto a largura da sépala, que é muito mais fraca e também negativa. Esta pode ser uma informação útil na fase posterior.
Olhando para a tabela de correlação, podemos ver que, em relação ao species_id, existem 3 variáveis principais (comprimento da sépala, comprimento da pétala e largura da pétala) que têm uma forte relação linear. Estas variáveis são provavelmente fortes variáveis na previsão das espécies de uma dada flor de íris.

-----


**Informações Visuais**

Agora que temos uma ideia do comportamento numérico dos dados, vamos dar uma olhada nos dados a partir de uma perspectiva visual.

Vamos traçar gráficos de cada uma das variáveis, umas contra as outras.
Como existem apenas 5 colunas numéricas, esse gráfico não será muito grande.

Para manter o tema das espécies, podemos codificar cada ponto de dados por suas respectivas espécies.


In [0]:
sns.set_style('whitegrid')
sns.pairplot(data, hue='species')
plt.show()

Isto nos mostra que as espécies estão bem separadas em grupos com base em suas medições. Este é o caso de todas as combinações, exceto ao traçar o comprimento da sépala e a largura da sépala. Estas são um pouco mais difíceis de separar.

Embora as espécies de Setosa neste gráfico estejam menos agrupadas do que nos outros gráficos, Versicolor e Virginica estão muito bem misturadas aqui, tornando muito mais difícil distinguir um grupo do outro individualmente.

--------------------------


In [0]:
sns.set_style('whitegrid')
plt.figure(figsize=(15,6))
mask = np.zeros_like(d_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(d_corr,mask=mask, cmap=cmap, vmax=0.99,
            square=True,
            linewidths=.5, cbar_kws={"shrink": .5})
plt.show()


Acima há uma representação visual da matriz de correlação mencionada anteriormente. Isso deixa muito claro que a largura da sépala é muito diferente das outras variáveis as quais tendem a variar de forma conjunta com uma forte correlação linear positiva.

------


**Agrupamentos através de K-Means**

A visualização anterior nos disse muito sobre os dados e sugere que espécies semelhantes se agrupam através das dimensões de suas pétalas e sépalas. 
Vamos explorar isso! A técnica de agrupamento usada para explorar isso será o agrupamento K-Means. K-Means, é uma técnica de aprendizado de máquina sem supervisão. Apenas para explicar, o aprendizado de máquina não supervisionado é um ramo que se concentra na identificação de tendências e padrões nos dados. 
O K-Means se concentra especificamente em definir o número "K" de clusters. A forma como  cada ponto é associado a um cluster se baseia na distância euclidiana do dado em relação ao ponto central do cluster. Assim, cada um dos pontos de dados é associado ao cluster mais próximo e, depois que todos os pontos tiverem sido atribuídos, teremos pontos de dados com características semelhantes agrupados em cada cluster. 
Dependendo de seus objetivos, você pode ter quantos clusters quiser. No entanto, quanto mais clusters, menor a probabilidade de fazer suposições generalizadas sobre os dados. 
Neste caso, como temos três espécies, vamos tentar usar três clusters com a dimensão sépala e pétala dos dados e ver o que parece!

------

**K-Means com K=3**

In [0]:
def cluster_1_label(alpha):
    if alpha == 0:
        return 'virginica'
    if alpha == 1:
        return 'setosa'
    if alpha == 2:
        return 'versicolor'

def cluster_2_label(beta):
    if beta == 1 or beta == 7 or beta == 8:
        return 'setosa'
    if beta == 0 or beta == 3 or beta == 6:
        return 'versicolor'
    if beta == 2 or beta == 4 or beta == 5:
        return 'virginica'


In [0]:
# KMeans Cluster to explore data - 3 Clusters
kmeans_model_1 = KMeans(n_clusters=3,random_state=123)

distances_1 = kmeans_model_1.fit_transform(data.iloc[:,0:4])

labels_1 = kmeans_model_1.labels_

data['cluster_1']=labels_1
data['cluster_1_label']=data['cluster_1'].apply(cluster_1_label)

with sns.color_palette("hls", 8):
    sns.pairplot(data.iloc[:,[0,1,2,3,-2]], hue='cluster_1')


Estes gráficos são muito parecidos com aqueles da sessão de "Informações Visuais".
Isto pode ser bom porque pressupõe que os dados estão agrupados de forma semelhante aos rótulos das espécies, mas conseguimos isto apenas usando suas dimensões de sépala e pétalas. 

Vamos dar uma olhada em como o K-Means com K=3 agrupou as amostras contra suas respectivas espécies.


In [0]:

pd.crosstab(data['species'], labels_1)

A tabela acima mostra que Setosa foi perfeitamente agrupada, no entanto, Versicolor e Virginica possuem um número de agrupamentos inconsistentes. Isso provavelmente se deve ao fato de a largura da sépala de ambas as espécies é relativamente indistinguível quando comparada ao comprimento da sépala. Talvez se houvesse mais clusters, eles poderiam ajudar a quebrar as áreas que estão misturadas entre as duas espécies e suas dimensões. 

Vamos tentar 9 clusters e ver que coisas interessantes podemos encontrar (nota: não foi utilizado nenhum método matemático para derivar o número de K, ele apenas foi escolhido aleatoriamente).

----

**K-Means com K=9**


In [0]:
# KMeans Cluster to explore data - 9 Clusters
kmeans_model_2 = KMeans(n_clusters=9,random_state=123)
distances_2 = kmeans_model_2.fit_transform(data.iloc[:,0:4])
labels_2 = kmeans_model_2.labels_
data['cluster_2']=labels_2
data['cluster_2_label']=data['cluster_2'].apply(cluster_2_label)
data = data.iloc[:,[0,1,2,3,4,5,6,7,8,9]]
sns.pairplot(data.iloc[:,[0,1,2,3,8]], hue='cluster_2')


Estes gráficos são um pouco mais difíceis de comparar com os gráficos originais, no entanto, é possível notar que há vários clusters distintos que se agrupam. 

Vamos dar uma olhada em como o K-Means com K=9 agrupou as amostras contra suas respectivas espécies.

In [0]:
pd.crosstab(data['species'],data['cluster_2'])

O K-Means com K=9 agrupou os dados muito melhor do que o modelo anterior. O número de amostras agrupadas incorretamente é muito menor que o do cluster anterior. Escolhendo as espécies com mais contagens no cluster como o cluster de espécies designado, podemos somar todos os agrupamentos e ver quantos foram incorretamente agrupados usando a espécie como referência.

In [0]:
cluster_1_accuracy = len(data[data['species']==data['cluster_1_label']])/len(data)
cluster_2_accuracy = len(data[data['species']==data['cluster_2_label']])/len(data)
print('K=3 KMeans -> {0:.4f}%'.format(cluster_1_accuracy*100))
print('K=9 KMeans -> {0:.4f}%'.format(cluster_2_accuracy*100))

Uma forma eficaz de comparar os resultados do modelo seria calcular a precisão de cada modelo. 

Para fazer isso, você adicionaria todas as observações corretamente previstas e as dividiria pelo número total de observações (ou seja, corretamente previsto + incorretamente previsto). Quando fazemos isso, vemos que o cluster com K = 3 teve uma precisão de 89,33%, enquanto que o cluster K = 9  apresentou a precisão de 96,67%, o que é melhor!

---

\
#Modelagem de Classificação Logística Multi-Classe

Vamos dar uma olhada na construção de um modelo que pode prever a qual espécie uma dada flor do gênero íris pertence com base nas dimensões de sua pétala e sépala.


\
Precisamos construir um modelo que possa tomar uma série de dados e produzir uma saída que nos indique quais espécies a flor possui em função das variáveis de entrada fornecidas. 

\
Conforme já foi visto anteriormente, este ramo do aprendizado de máquina é chamado de aprendizado de máquina supervisionado.

\
Este é um ramo da aprendizagem de máquina que se concentra em treinar um algoritmo para prever uma variável desconhecida a partir de variáveis conhecidas. 

\
A variável desconhecida que preveremos é a espécie da flor. O algoritmo que treinaremos é uma Regressão Logística.

\
Uma vez que tentaremos prever as espécies de flores (que é representado por uma lista finita de opções categóricas - Setosa, Virginica, Versicolor), precisamos atribuir um valor numérico para cada uma das espécies dentro de variáveis dummy. 

\
Variáveis dummy são variáveis que recebem atribuições binárias (True = 1, False = 0) com base em cada categoria em uma determinada coluna. 

\
Por exemplo, nossas categorias são Setosa, Virginica e Versicolor. Nós teremos uma coluna para cada uma dessas espécies e atribuiremos 1 ou 0 para cada linha, dependendo de quais espécies elas pertencem. Portanto, uma linha da amostra representando Setosa terá o valor de coluna Setosa=1 e os valores Virginica e Versicolor serão designados como 0.



In [0]:
dummies = pd.get_dummies(data['species'],prefix='actual')
data = pd.concat([data,dummies],axis=1)

In [0]:
data.head()

Aqui, as variáveis dummy são facilmente criadas e concatenadas aos dados principais graças aos métodos realmente úteis do Pandas.

A forma como uma Regressão Logística funciona é muito semelhante a uma regressão linear, pois é preciso um número de entradas para fornecer uma saída, no entanto, a saída é onde está a diferença, pois usa a "logit transformation", que é uma mistura de uma transformação exponencial e uma transformação de normalização.

A Regressão Logística é um modelo de classificação que gera a probabilidade de você estar em uma determinada classe. O limite para determinar qual probabilidade deve ser atribuída a um retorno positivo ou negativo é padronizado para 0,5 e pode ser ajustado dependendo de suas necessidades. Isso também é conhecido como o limiar de discriminação.

Estes modelos estão focados em saídas binárias. No nosso caso, temos mais de duas saídas; temos 3 espécies para explicar. Então, para fazer isto, teremos que construir 3 modelos separados para cada uma das diferentes espécies. Cada modelo irá prever a chance de uma determinada entrada ser classificada como essa espécie ou não. Para cada linha, cada modelo calculará sua probabilidade e a mais alta das 3 será a espécie selecionada para aquela linha.

In [0]:
classifiers = ['versicolor','virginica','setosa']
models = {}
for mdl_idx in classifiers:
    lgr_model = LogisticRegression()
    lgr_model.fit(data.iloc[:,[0,1,2,3]],data['actual_{}'.format(mdl_idx)])
    models[mdl_idx]=lgr_model
models

Os modelos foram montados e treinados utilizando todo o conjunto de dados e é assim que eles são configurados. Eles foram agrupados em um dicionário para que possam ser acessados através de um loop de forma segura.


In [0]:

coefs = {}
for mdl_k, mdl_v in models.items():
    coefs[mdl_k]=mdl_v.coef_
coefs

Vamos dar uma olhada nos coeficientes do modelo para determinar como cada uma das variáveis afeta a variável dependente, que neste caso é a probabilidade de pertencer a uma determinada espécie. 

Aqui podemos ver o efeito que cada aumento de dimensão tem na probabilidade de ser classificado nas respectivas espécies. Cada coeficiente (k) representa o efeito (+ ve ou -ve) para cada aumento de 1 unidade na variável à qual eles estão atribuídos sobre a variável dependente.

In [0]:
lgr_probabilities = pd.DataFrame(columns=classifiers)
for mdl_key, mdl_model in models.items():
    lgr_probabilities[mdl_key] = mdl_model.predict_proba(data.iloc[:,[0,1,2,3]])[:,1]
lgr_probabilities.head()

Aqui nós calculamos as probabilidades para cada um dos modelos para que possamos determinar qual linha pertence a qual espécie de acordo com o modelo com maior probabilidade.

In [0]:
predicted_species = lgr_probabilities.idxmax(axis=1)
pred_spec = lgr_probabilities.max(axis=1)
lgr_accuracy = len(data[data['species']==predicted_species])/len(data)
print("Accuracy", lgr_accuracy)
pd.crosstab(data['species'],predicted_species)


Aqui precisamos selecionar o maior valor em cada linha e usar o nome do índice da coluna para sabermos quais espécies o modelo prevê ter maior probabilidade de ser usada. Esta tabela mostra como as previsões se comparam aos valores reais.

In [0]:
print('Logistic Regression Accuracy - {0:.4f}%'.format(lgr_accuracy*100));
print('3-Cluster K-Means Clustering Accuracy - {0:.4f}%'.format(cluster_1_accuracy*100));
print('9-Cluster K-Means Clustering Accuracy - {0:.4f}%'.format(cluster_2_accuracy*100));

A acurácia deste modelo pode ser calculada da mesma forma que os modelos K-Means anteriores. 

Este cálculo resulta em uma precisão de 96,00%! 

Isto é apenas 0,67% menor do que o modelo K-Means dom K=9.

---

Vamos examinar um pouco mais a fundo cada um dos modelos usados para fazer essas previsões, para avaliarmos sua precisão.


In [0]:
log_setosa_fpr, log_setosa_tpr, log_setosa_thresholds = roc_curve(data['actual_setosa'],
                                                                  lgr_probabilities['setosa'])
log_versicolor_fpr, log_versicolor_tpr, log_versicolor_thresholds = roc_curve(data['actual_versicolor'],
                                                                              lgr_probabilities['versicolor'])
log_virginica_fpr, log_virginica_tpr, log_virginica_thresholds = roc_curve(data['actual_virginica'],
                                                                           lgr_probabilities['virginica'])


As duas métricas mais populares usadas na avaliação de modelos de classificação são a Taxa de Positivos Verdadeiros (TPR) e a Taxa de Falsos Positivos (FPR) do modelo. A taxa positiva real fornece informações sobre a capacidade do modelo de atribuir corretamente resultados positivos. Isso também é chamado de "sensibilidade" do modelo. A Taxa de Falsos Positivos fornece informações sobre a proporção de resultados que previmos como positivos quando eram negativos. Isso também é chamado de fallout do modelo.

Quando você planeja essas duas medidas uma contra a outra, isso produz um gráfico conhecido como Curva ROC (Receiver Operator Characteristic). Essa curva nos permite entender o desempenho de um modelo de classificação à medida que o limiar de discriminação é variado. 

Ainda mais, a área sob a curva descreve a probabilidade de o classificador classificar uma observação positiva aleatória maior do que uma observação negativa aleatória. Isso é conhecido como a pontuação AUC (Área sob Curva). 

Vamos dar uma olhada nas Curvas ROC dos 3 modelos e no que eles poderiam nos dizer sobre cada modelo.

Em cada gráfico, a linha pontilhada cinza indica a mesma probabilidade de predição (ou seja, 50/50).


In [0]:
plt.figure(figsize=(15,6))
plt.xlim([-0.2, 1.2])
plt.ylim([0, 1.2])
plt.title('Setosa Model ROC Curve')
plt.xlabel('log_setosa_fpr')
plt.ylabel('log_setosa_tpr')
plt.plot(log_setosa_fpr, log_setosa_tpr,c='r')
plt.plot([0,1],[0,1], c='grey',ls=':')
plt.show()

In [0]:
plt.figure(figsize=(15,6))
plt.xlim([-0.2, 1.2])
plt.ylim([0, 1.2])
plt.title('Versicolor Model ROC Curve')
plt.xlabel('log_versicolor_fpr')
plt.ylabel('log_versicolor_tpr')
plt.plot(log_versicolor_fpr, log_versicolor_tpr,c='g')
plt.plot([0,1],[0,1], c='grey',ls=':')
plt.show()

In [0]:
plt.figure(figsize=(15,6))
plt.xlim([-0.2, 1.2])
plt.ylim([0, 1.2])
plt.title('Virginica Model ROC Curve')
plt.xlabel('log_virginica_fpr')
plt.ylabel('log_virginica_tpr')
plt.plot(log_virginica_fpr, log_virginica_tpr,c='b')
plt.plot([0,1],[0,1], c='grey',ls=':')
plt.show()

In [0]:
log_setosa_auc = roc_auc_score(data['actual_setosa'],lgr_probabilities['setosa'])
log_versicolor_auc = roc_auc_score(data['actual_versicolor'],lgr_probabilities['versicolor'])
log_virginica_auc = roc_auc_score(data['actual_virginica'],lgr_probabilities['virginica'])

print('The AUC score for Sentosa Model - {0:.4f}%'.format(log_setosa_auc*100))
print('The AUC score for Versicolor Model - {0:.4f}%'.format(log_versicolor_auc*100))
print('The AUC score for Virginica Model - {0:.4f}%'.format(log_virginica_auc*100))

---

**K-Fold Cross Validation**

Estes modelos foram construídos a partir de todo o conjunto de dados e podem estar sujeitos a overfitting. 

O overfitting ocorre quando um modelo é bom apenas para prever os dados nos quais foi treinado, e o algoritmo não fornece uma solução generalizada suficiente que possa ser aplicada a dados ainda não vistos. 

Overfitting é um problema se você está construindo seu modelo para fazer previsões sobre dados sobre os quais ele nunca foi treinado. 

Para superar esse grande desafio, usaremos uma técnica de validação cruzada chamada"K-Fold Cross Validation".

A validação cruzada supera o overfitting ao sub-definir os dados em um conjunto de treinamento e um conjunto de testes. 
Ao fazer isso, os modelos são treinados no conjunto de treinamento e testados usando o conjunto de testes que representaria os dados desconhecidos que o modelo está tentando prever. 

A K-Fold Cross Validation está basicamente repetindo o processo de validação cruzada K vezes.

Cada iteração teria um treinamento e um conjunto de testes diferentes. Depois de executar K vezes, a precisão e as métricas médias são tomadas e transportadas como a precisão final e a métrica para a técnica de modelagem.

In [0]:
# k fold cross validation of results
kf = KFold(len(data), n_folds=10, shuffle=True, random_state=123)
classifiers = ['versicolor','virginica','setosa']
k_models = {}
k_probabilities = {}
k_accuracies = {}
k_pred_spec = {}
k_roc = {}
k_auc = {}
colors = ['m','y','k','#9500d8','#a6ff9b','#7f3a18','b','g','r','c']

for mdl_idx, (train_idx, test_idx) in enumerate(kf):
    train_data = data.iloc[train_idx]
    test_data = data.iloc[test_idx]
    models = {}
    
    for classer in classifiers:
        model = LogisticRegression()
        model.fit(train_data.iloc[:,[0,1,2,3]],train_data['actual_{}'.format(classer)])
        models[classer]=model
    k_models[mdl_idx]=models
    temp_probabilities = pd.DataFrame(columns=classifiers)
    
    for mdl_key, mdl_model in k_models[mdl_idx].items():
        temp_probabilities[mdl_key] = mdl_model.predict_proba(test_data.iloc[:,[0,1,2,3]])[:,1]
        k_probabilities[mdl_idx]=temp_probabilities

    for mdl_key, mdl_probs in k_probabilities[mdl_idx].items():
        predicted_species = temp_probabilities.idxmax(axis=1)
        pred_spec=temp_probabilities.max(axis=1)
        #lgr_accuracy = len(test_data[test_data['species']==predicted_species])/len(test_data)
        df1 = test_data['species'].reset_index(drop=True)
        lgr_accuracy = len(df1==predicted_species)/len(test_data)
        k_accuracies[mdl_idx]=lgr_accuracy
        k_pred_spec[mdl_idx]=pred_spec
    
    roc={}
    for classer in classifiers:
        fpr, tpr, thresholds = roc_curve(test_data['actual_{}'.format(classer)],k_probabilities[mdl_idx][classer])
        roc[classer]=(fpr,tpr,thresholds)
    k_roc[mdl_idx]=roc
    
    auc={}
    for classer in classifiers:
        auc_score = roc_auc_score(test_data['actual_{}'.format(classer)],k_probabilities[mdl_idx][classer])
        auc[classer]=auc_score
    k_auc[mdl_idx]=auc

In [0]:
for classer in classifiers:
    plt.figure(figsize=(15,6))
    plt.xlim([-0.2, 1.2])
    plt.ylim([0, 1.2])
    plt.title('{} Model ROC Curve'.format(classer))
    plt.xlabel('{} False Positive Rate'.format(classer))
    plt.ylabel('{} True Positive Rate'.format(classer))
    for k, v in k_roc.items():
        fpr = v[classer][0]
        tpr = v[classer][1]
        plt.plot(fpr, tpr, c=colors[k])
    plt.plot([0,1],[0,1], c='grey',ls=':')
    plt.show()

In [0]:
k_auc

Aqui a técnica de validação cruzada mencionada anteriormente é executada usando K = 10. 

Isso significa que o processo de modelagem, previsão e pontuação foi executado 10 vezes. Um total de 3 * K = 10 (30) modelos foram construídos e testados.


Os gráficos mostram que os modelos Setosa foram consistentes em predizer quais das amostras eram da espécie Setosa. A pontuação média da AUC para os modelos Setosa foi de 100,00%. 

Os modelos da Virginica só tinha um conjunto em que o modelo tinha um pequeno problema em predizer as espécies corretas, no entanto, a pontuação média da AUC foi de 99,63%. 

Os modelos Versicolor tiveram mais problemas em prever as espécies de cada amostra com pontuações AUC variando de 59,09% a 94,44%. A pontuação média da AUC para os modelos Versicolor foi de 80,04%. 

Com base nas pontuações da AUC, o grupo de modelos com melhor desempenho foi o grupo 3 com pontuações AUC de Setosa=100%, Versicolor=94,44% e Virginica=100,00%.


In [0]:
setosa_auc = 0
versicolor_auc = 0
virginica_auc = 0

for k,v in k_auc.items():
    setosa_auc += v['setosa']
    versicolor_auc += v['versicolor']
    virginica_auc += v['virginica']

print('The Cross Validated AUC score for the Sentosa Model - {0:.4f}%'.format(setosa_auc*10))
print('The Cross Validated AUC score for the Versicolor Model - {0:.4f}%'.format(versicolor_auc*10))
print('The Cross Validated AUC score for the Virginica Model - {0:.4f}%'.format(virginica_auc*10))

***Conclusão***

Após a conclusão de nossa análise, agora podemos avaliar o quanto atingimos cada um de nossos objetivos e responder às perguntas que geraram as várias análises.

*Objetivos*

1. Verifique se há clusters identificáveis nos dados: Conseguimos usar o agrupamento K-Means para identificar padrões e o algoritmo tendia a agrupar os dados com base em suas espécies.

2. Avaliar a precisão desses agrupamentos com base nas espécies: Pudemos avaliar a precisão dos métodos de agrupamento observando o número de pontos de dados corretamente categorizados em relação à toda a população.

3. Construir um modelo para prever as espécies de amostras: Conseguimos construir um modelo preditivo para determinar as espécies das flores. O melhor moelo foi o modelo de regressão logística. Nós construímos um para cada espécie e usamos uma abordagem One Vs All.

4. Avaliar a precisão das previsões do modelo com base nas espécies reais: Conseguimos avaliar a precisão dos modelos usando a mesma abordagem dos métodos de agrupamento.

5. Avaliar a capacidade do modelo de prever cada espécie: Conseguimos atingir esse objetivo mergulhando cada vez mais fundo no modelo de cada espécie e calculando cada uma das suas pontuações na AUC.

6. Avaliar a precisão do modelo para prever dados fora da amostra: Conseguimos avaliar a precisão do modelo em dados fora da amostra simulando isso com uma técnica de K-Fold Cross Validation.


Também estamos em uma ótima posição para responder a todas as questões devido às descobertas em nossas análises.

Questões de pesquisa
1. Quais das duas abordagens são mais precisas? - Tendo utilizado medidas de precisão adequadas, podemos concluir que o cluster K-Means com K=9 foi o classificador mais preciso com uma pontuação de precisão de 96,67%.

2. Como cada uma das variáveis interage umas com as outras? - Conseguimos identificar as relações entre as variáveis com o uso de uma matriz de correlação que nos permitiu identificar que 3 de 4 variáveis são altamente correlacionadas entre si e com a espécie.

3. Como cada uma das variáveis influencia a previsão da espécie? - Examinamos os coeficientes de cada um dos modelos usados para determinar isto.

4. Qual espécie é mais fácil de prever? Por quê? - Com base em cada uma das curvas ROC do modelo e nos modelos K-Means, a espécie mais fácil de prever é Setosa e isso se deve às características distintas da espécie que é bastante diferente das duas espécies restantes.

5. Qual espécie é a mais difícil de prever? Por quê? - A espécie mais difíceis de prever com base na curva ROC e nos resultados da AUC é a espécie Versicolor. Seus atributos são muito semelhantes às espécies de Virginica, tornando muito difícil distingui-los uns dos outros. Isso fica claro em algumas parcelas e agrupamentos em que há uma quantidade razoável de sobreposição, mesmo no cluster K = 9. A espécie Versicolor é também está posicionada entre as duas outras espécies, tornando mais difícil definir um intervalo entre elas que não se enquadre em nenhuma outra espécie
